In [1]:
from time import sleep
import pandas as pandas
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
import pandas as pd



In [2]:
url = 'https://www.yelp.fr/biz/le-dépanneur-pigalle-paris-3'


In [3]:
driver = webdriver.Chrome('/Users/admin/chromedriver')
driver.get(url)
sleep(10)


comment_blocs = driver.find_elements_by_css_selector('.review__373c0__13kpL.border-color--default__373c0__3-ifU')


In [65]:
comment_blocs = driver.find_elements_by_css_selector('.review__373c0__13kpL.border-color--default__373c0__3-ifU')
detailed_reviews_list=[]
 
for i in range(len(comment_blocs)):
    try:
        date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[3].text.split()[3]
    except: 
        date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[3].text.split()[2]
    rating = comment_blocs[i].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label')[0]

    username = comment_blocs[i].find_elements_by_css_selector('a.link__373c0__1G70M.link-size--inherit__373c0__1VFlE')[0].text
    
    try:
        photos_for_review = comment_blocs[i].find_elements_by_css_selector('a.link__373c0__1G70M.link-size--inherit__373c0__1VFlE')[1].text[0]
    except:
        photos_for_review = 0
    user_location =  comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--normal__373c0__3xep9.text-align--left__373c0__2XGa-')[0].text
    text_review =  comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--normal__373c0__3xep9.text-align--left__373c0__2XGa-')[1].text
    friends_count = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-weight--semibold__373c0__2l0fe')[0].text
    total_reviews_of_user = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-weight--semibold__373c0__2l0fe')[1].text

    try:
        total_image_posted = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-weight--semibold__373c0__2l0fe')[2].text
    except:
        total_image_posted = 0

    is_real_review = 1

    detailed_reviews_list.append([date, username, photos_for_review, rating, text_review, user_location, friends_count, total_reviews_of_user, total_image_posted, is_real_review])

    
    print(i, 'OK')
df = pd.DataFrame(detailed_reviews_list,
                             columns = ['date','username', 'photos_for_review', 'rating',
                                         'text_review', 'user_location', 'friends_count',
                                          'total_reviews_of_user', 'total_image_posted', 'is_real_review'])


0 OK
1 OK
2 OK
3 OK
4 OK
5 OK
6 OK
7 OK
8 OK
9 OK
10 OK
11 OK
12 OK
13 OK
14 OK


IndexError: list index out of range

In [32]:
df

,username,photos_for_review,rating,text_review,city_of_user,number_of_friends,total_reviews_of_user,total_image_posted,is_real_review
0,Olivier A.,2,4,Très bel établissement dans le 9eme branché et...,Paris,50,52,67,1
1,Noémie B.,4,4,"Alors alors, le Dépanneur, si j'y mets 4 étoil...",Paris,96,285,486,1
2,Tanguy L.,0,2,Guacamole plein d'eau belle présentation mais ...,Paris,5,16,7,1
3,Louise M.,0,3,"Ce lieu est vraiment bien, les serveurs et ser...",Paris,83,93,284,1
4,Pierre-morgan A.,1,4,"Ambiance et cadre très sympa, situé dans un qu...","Barbès/Goutte d'Or, Paris",92,12,14,1
5,Thibault N.,1,4,"Les burgers ne sont pas donnés, mais très bons...",Paris,10,52,35,1
6,Pierre B.,1,5,Je suis conquis :)\nJe crois que c'est mon pre...,Paris,2,20,11,1
7,Aintsoa R.,1,3,"La décoration: tout mignon tout joli, du bois,...",Paris,9,14,11,1
8,Marine B.,1,4,Rectificatif sur les burgers signés Cantine Ca...,Paris,671,198,441,1
9,Noemie T.,1,3,"Rien que pour le bar! Le mieux, c'est de laiss...",Paris,3,6,3,1


In [281]:
comment_blocs[20].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label')[0]

IndexError: list index out of range

In [61]:
for i in range(20):
    print(i, comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[-1].text.split()[-1])

0 25/10/2017
1 10/06/2016
2 07/06/2018
3 08/07/2017
4 05/06/2016
5 23/02/2015
6 28/08/2015
7 16/06/2014
8 précédent
9 29/10/2014
10 31/07/2013
11 12/09/2013
12 30/07/2013
13 21/08/2016
14 29/04/2015
15 22/04/2015
16 27/01/2014
17 28/05/2015
18 23/09/2015
19 24/08/2015


In [64]:
for i in range(7):
    print(comment_blocs[8].find_elements_by_css_selector('div.review__373c0__13kpL span.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[i].text)

671
198
441
Publié le 07/07/2014
Avis mis à jour
Publié le 13/01/2014
Avis précédent
